In [1]:
PROJECT = "area51-lab"
DATASET = "raw"
TABLE = "test"
TABLE_PATH = f"projects/{PROJECT}/datasets/{DATASET}/tables/{TABLE}"
WRITE_STREAM = f"{TABLE_PATH}/streams/_default"

%env GOOGLE_APPLICATION_CREDENTIALS=/Users/klim/Desktop/area51.json

env: GOOGLE_APPLICATION_CREDENTIALS=/Users/klim/Desktop/area51.json


In [3]:
!./protoc --python_out=. bqmodel.proto

In [2]:
from datetime import datetime as dt, timezone as tz
from google.cloud.bigquery_storage import (
    BigQueryWriteClient,
    AppendRowsRequest, 
    WriteStream,
    ProtoRows,
    ProtoSchema
)
from google.protobuf import descriptor_pb2

import bqmodel_pb2 as model

client = BigQueryWriteClient()
parent = client.table_path(PROJECT, DATASET, TABLE)
write_stream = client.create_write_stream(
    parent=parent, 
    write_stream=WriteStream(type_=WriteStream.Type.COMMITTED)
)

write_stream.name

'projects/area51-lab/datasets/raw/tables/test/streams/CiQ2ZmEwMzA4OC0wMDAwLTI0YzYtYTA2MS0yNDA1ODg3MWZjZDg'

In [3]:
proto_descriptor = descriptor_pb2.DescriptorProto()
model.Tweet.DESCRIPTOR.CopyToProto(proto_descriptor)

delta = dt.now() - dt(1970, 1, 1)
ts_value = int(delta.total_seconds()) * 1000000 + int(delta.microseconds)

row = model.Tweet()
row.id = "123123132131"
row.url = "123123132131"
row.content = "123123132131"
row.created_timestamp = ts_value
row.user_id = "123123132131"
row.retweets_count = 1
row.quote_tweets_count = 1
row.likes_count = 1
row.hashtags = []
row.replies.extend(["123123132131", "123123132131"])
row.source_label = "123123132131"

proto_schema = ProtoSchema(proto_descriptor=proto_descriptor)
proto_rows = ProtoRows(serialized_rows=[
    row.SerializeToString(),
    row.SerializeToString(),
    row.SerializeToString()
])
proto_data = AppendRowsRequest.ProtoData(
    writer_schema=proto_schema,
    rows=proto_rows
)

request = AppendRowsRequest(write_stream=write_stream.name, proto_rows=proto_data)

AttributeError: Assignment not allowed to repeated field "hashtags" in protocol message object.

In [6]:
resp = client.append_rows(requests=iter([request]))
resps = list(resp)
resps

[append_result {
   offset {
     value: 3
   }
 }]

In [7]:
for resp in resps:
    print(resp.append_result)
    if resp.append_result:
        pass

offset {
  value: 3
}

